# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps 
import os
import json
from pandas.io.json import json_normalize
from pprint import pprint
from ipywidgets.embed import embed_minimal_html



# Import API key
from api_keys import g_key 




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
#Load the CSV file from previous project
#set a variable for the file location
CityData=pd.read_csv("../output_data/CityDataStructured.csv")
CityData.head()




,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Kavaratti,20,IN,1581142839,58,10.57,72.64,86.00,4.70
1,1,Puerto Ayora,40,EC,1581142839,86,-0.74,-90.35,77.88,8.70
2,2,Kahului,20,US,1581142840,64,20.89,-156.47,69.80,5.82
3,3,Nome,1,US,1581142840,84,64.50,-165.41,-29.00,6.22
4,4,Antalaha,0,MG,1581142840,74,-14.90,50.28,80.73,12.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
#Access maps with unique API key
gmaps.configure(api_key=g_key)

In [35]:

# Store 'Lat' and 'Lng' into  locations 
locations = CityData[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#CityData= CityData.dropna()
HumidityValue = CityData["Humidity"].astype(float)
maxHumidity=CityData["Humidity"].max()




### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
CityDatanew=CityData.dropna(how="all")

In [37]:
# Favourite place anf direction
MyChoicePlaces=CityDataFresh.loc[(CityDataFresh["Cloudiness"] == 0) & (CityDataFresh["Wind Speed"] < 10) & ((CityDataFresh["Max Temperature"] >70 ) & (CityDataFresh["Max Temperature"] <= 80))]
MyChoicePlaces

,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Cloud Feel,Temp Feel,Wind Feel
31,31,Tura,0,IN,1581142844,32,25.52,90.22,77.92,7.38,Favorable,Favorable,Favorable
44,44,Hermanus,0,ZA,1581142846,81,-34.42,19.23,72.00,7.00,Favorable,Favorable,Favorable
93,93,Cape Town,0,ZA,1581142802,68,-33.93,18.42,72.00,6.93,Favorable,Favorable,Favorable
107,107,Golakganj,0,IN,1581142858,39,26.10,89.83,74.50,8.97,Favorable,Favorable,Favorable
144,144,Puerto Baquerizo Moreno,0,EC,1581142864,86,-0.90,-89.60,78.78,5.59,Favorable,Favorable,Favorable
148,148,Baharly,0,TM,1581142865,16,38.44,57.43,74.64,9.26,Favorable,Favorable,Favorable
176,176,Ndele,0,CF,1581142869,23,8.41,20.65,74.44,5.55,Favorable,Favorable,Favorable
218,218,Sao Joao da Barra,0,BR,1581142878,88,-21.64,-41.05,75.87,7.45,Favorable,Favorable,Favorable
243,243,Champerico,0,GT,1581142881,82,14.30,-91.92,74.41,4.25,Favorable,Favorable,Favorable
249,249,Mecca,0,SA,1581142882,63,21.43,39.83,71.69,1.61,Favorable,Favorable,Favorable


In [38]:

###### Step 2 ###### create Binning ###########
WeatherSituation_Bin_Build=[CityDataFresh["Cloudiness"].describe()["min"],0.1,
                            CityDataFresh["Cloudiness"].describe()["25%"],CityDataFresh["Cloudiness"].describe()
                            ["50%"],CityDataFresh["Cloudiness"].describe()["75%"],CityDataFresh["Cloudiness"].describe()["max"]]
WeatherSituation_Bin =["Favorable","Clear Sky","Mild Cloud","Mostly Cloudy","Totally Cloudy"]
Temp_Bin_Build=[CityDataFresh["Max Temperature"].describe()["min"],70,80,
                CityDataFresh["Max Temperature"].describe()["max"]]
Temp_Bin=["Cold","Favorable","Hot"]
Wind_Bin_Build=[CityDataFresh["Wind Speed"].describe()["min"],10,
                CityDataFresh["Wind Speed"].describe()["max"]]
Wind_Bin=["Favorable","Windy"]
CityData_Favor = CityDataFresh
CityData_Favor["Cloud Feel"] = pd.cut(CityData_Favor["Cloudiness"], 
                                      WeatherSituation_Bin_Build ,right=True,labels=WeatherSituation_Bin,include_lowest=True)
CityData_Favor["Temp Feel"]= pd.cut(CityData_Favor["Max Temperature"], 
                                    Temp_Bin_Build ,right=True,labels=Temp_Bin,include_lowest=True)
CityData_Favor["Wind Feel"]= pd.cut(CityData_Favor["Wind Speed"], 
                                    Wind_Bin_Build ,right=True,labels=Wind_Bin,include_lowest=True)
CityData_Favor.sort_values(['Wind Speed'], ascending=[1])
BestWeatherCities=CityData_Favor.loc[(CityData_Favor["Wind Feel"] == "Favorable") & (CityData_Favor["Temp Feel"] == "Favorable") & (CityData_Favor["Cloud Feel"] == "Favorable")]
len(BestWeatherCities)

19

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:

hotel_df=MyChoicePlaces


In [40]:

HotelName = []
HotelAddress=[]
# function to extract hotel name
def GetHotelName(Latitude,Longitude):
    TargetRadius = 5000
    TargetType = "Hotel"
    TargetCoordinates =  Latitude + " , " + Longitude 
    params = {
    "location": TargetCoordinates,
    "radius": TargetRadius,
    "type": TargetType,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    resp = requests.get(base_url, params=params)
    rtext = json.loads(resp.text)
    response = json_normalize(rtext)
    ResultedHotels=json_normalize(response["results"][0])
    HotelName.append(ResultedHotels["name"][1])
    HotelAddress.append(ResultedHotels["vicinity"][1])

In [41]:

for index, row in BestWeatherCities.iterrows():
    GetHotelName(str(row['Latitude']),str(row['Longitude']))

In [42]:
HotelName,HotelAddress

(['Click N Browse Internet Cafe, Sukbhir Bhawan, Tura',
  'Misty Waves Boutique Hotel',
  'Southern Sun Waterfront Cape Town',
  'South Raipur Sub Center',
  'Hotel los Algarrobos',
  'TM CELL',
  'IOM OFFICE',
  'AC São João da Barra',
  'Internet y Compuaccesorios "Fastnet"',
  'Hotel Fairmont Makkah Clock Royal Tower',
  'Port Lincoln Hotel',
  "Jimmi Girl's Government Secondary School",
  'INN NEW HORIZON',
  'Bolan Engineering Workshop',
  'Swan Express Rio Grande',
  'IPS-Puesto Sanitario Fuerte Olimpo',
  'Ersal Money Transfer',
  'Ecobank - Gboko Branch, Benue State',
  'Anamis Photographic'],
 ['Bazaar - Hawakhana Road, Tura',
  '21 Marine Drive, Hermanus',
  '1 Lower Buitengracht, Cape Town City Centre, Cape Town',
  'National Highway 31, Batuatuli',
  'Av Quito Y, Avenida Alsacio Northia, Puerto Baquerizo Moreno',
  'M37, Baharly',
  "N'Délé",
  'Rua do Sacramento, 116 - Centro, São João da Barra',
  'Champerico',
  'King Abdul Aziz Endowment, Ajyad Street, Mecca',
  '1 Linc

In [43]:

hotel_df.insert(2, "Hotel Name", HotelName, True) 
hotel_df.insert(3, "Hotel Address", HotelAddress, True)
hotel_df

,City ID,City Name,Hotel Name,Hotel Address,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Cloud Feel,Temp Feel,Wind Feel
31,31,Tura,"Click N Browse Internet Cafe, Sukbhir Bhawan, ...","Bazaar - Hawakhana Road, Tura",0,IN,1581142844,32,25.52,90.22,77.92,7.38,Favorable,Favorable,Favorable
44,44,Hermanus,Misty Waves Boutique Hotel,"21 Marine Drive, Hermanus",0,ZA,1581142846,81,-34.42,19.23,72.00,7.00,Favorable,Favorable,Favorable
93,93,Cape Town,Southern Sun Waterfront Cape Town,"1 Lower Buitengracht, Cape Town City Centre, C...",0,ZA,1581142802,68,-33.93,18.42,72.00,6.93,Favorable,Favorable,Favorable
107,107,Golakganj,South Raipur Sub Center,"National Highway 31, Batuatuli",0,IN,1581142858,39,26.10,89.83,74.50,8.97,Favorable,Favorable,Favorable
144,144,Puerto Baquerizo Moreno,Hotel los Algarrobos,"Av Quito Y, Avenida Alsacio Northia, Puerto Ba...",0,EC,1581142864,86,-0.90,-89.60,78.78,5.59,Favorable,Favorable,Favorable
148,148,Baharly,TM CELL,"M37, Baharly",0,TM,1581142865,16,38.44,57.43,74.64,9.26,Favorable,Favorable,Favorable
176,176,Ndele,IOM OFFICE,N'Délé,0,CF,1581142869,23,8.41,20.65,74.44,5.55,Favorable,Favorable,Favorable
218,218,Sao Joao da Barra,AC São João da Barra,"Rua do Sacramento, 116 - Centro, São João da B...",0,BR,1581142878,88,-21.64,-41.05,75.87,7.45,Favorable,Favorable,Favorable
243,243,Champerico,"Internet y Compuaccesorios ""Fastnet""",Champerico,0,GT,1581142881,82,14.30,-91.92,74.41,4.25,Favorable,Favorable,Favorable
249,249,Mecca,Hotel Fairmont Makkah Clock Royal Tower,"King Abdul Aziz Endowment, Ajyad Street, Mecca",0,SA,1581142882,63,21.43,39.83,71.69,1.61,Favorable,Favorable,Favorable


In [44]:
# Create a Hotels
# Store 'Lat' and 'Lng' into  locations 
hlocations = hotel_df[["Latitude", "Longitude"]].astype(float)
HumidityValue = CityData["Humidity"].astype(float)
maxHumidity=CityData["Humidity"].max()

myhotellocator = gmaps.figure(center=(20.0, 0.0), zoom_level=2)

HotelHeatLayer = gmaps.heatmap_layer(locations, weights=HumidityValue, 
                                 dissipating=False, max_intensity=maxHumidity,
                                 point_radius = 1)

myhotellocator.add_layer(HotelHeatLayer)
myhotellocator



Figure(layout=FigureLayout(height='420px'))

In [45]:
# Add marker layer ontop of heat map
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
MyInfoTemplate = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hover=np.array(hotel_df["Hotel Name"])
#locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map
Hover=np.array(hotel_df["City Name"])
HotelMarker_Layer=gmaps.marker_layer(hlocations,hover_text=hover,info_box_content=MyInfoTemplate,display_info_box=True)
myhotellocator.add_layer(HotelMarker_Layer)
#Save the Map
embed_minimal_html('../output_data/GMAP_HotelMarkerOnHeatMap.html', views=[myhotellocator])
# Display Map
myhotellocator

Figure(layout=FigureLayout(height='420px'))